# Cartpole DQN

Deep Q-Learning Network with Keras and OpenAI Gym, based on [Keon Kim's code](https://github.com/keon/deep-q-learning/blob/master/dqn.py).

#### Import dependencies

In [1]:
import random
import gymnasium as gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import os # for creating directories

2024-01-15 09:34:23.326320: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-15 09:34:23.553909: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-15 09:34:23.554839: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-15 09:34:24.422855: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


#### Set parameters

In [2]:
env = gym.make('CartPole-v1') # initialise environment

In [3]:
state_size = env.observation_space.shape[0]
state_size

4

In [4]:
action_size = env.action_space.n
action_size

2

In [5]:
batch_size = 32

In [6]:
n_episodes = 1001 # n games we want agent to play (default 1001)

In [7]:
output_dir = 'model_output/cartpole/'

In [8]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

#### Define agent

In [9]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000) # double-ended queue; acts like list, but elements can be added/removed from either end
        self.gamma = 0.95                # decay or discount rate: enables agent to take into account future actions in addition to the immediate ones, but discounted at this rate
        self.epsilon = 1.0               # exploration rate: how much to act randomly; more initially than later due to epsilon decay
        self.epsilon_decay = 0.995       # decrease number of random explorations as the agent's performance (hopefully) improves over time
        self.epsilon_min = 0.01          # minimum amount of random exploration permitted
        self.learning_rate = 0.001       # rate at which NN adjusts models parameters via SGD to reduce cost 
        self.model = self._build_model() # private method 
    
    def _build_model(self):
        # neural net to approximate Q-value function:
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu')) # 1st hidden layer; states as input
        model.add(Dense(24, activation='relu'))                            # 2nd hidden layer
        model.add(Dense(self.action_size, activation='linear'))            # 2 actions, so 2 output neurons: 0 and 1 (L/R)
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done)) # list of previous experiences, enabling re-training later

    def act(self, state):
        if np.random.rand() <= self.epsilon:   # if acting randomly, take random action
            return random.randrange(self.action_size)
        act_values = self.model.predict(state, verbose=0) # if not acting randomly, predict reward value based on current state
        return np.argmax(act_values[0])        # pick the action that will give the highest reward (i.e., go left or right?)

    def replay(self, batch_size): # method that trains NN with experiences sampled from memory
        minibatch = random.sample(self.memory, batch_size)         # sample a minibatch from memory
        for state, action, reward, next_state, done in minibatch:  # extract data for each minibatch sample
            target = reward  # if done (boolean whether game ended or not, i.e., whether final state or not), then target = reward
            if not done:     # if not done, then predict future discounted reward
                target = (reward + self.gamma *     # (target) = reward + (discount rate gamma) * 
                          np.amax(self.model.predict(next_state, verbose=0)[0])) # (maximum target Q based on future action a')
            target_f = self.model.predict(state, verbose=0)    # approximately map current state to future discounted reward
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0) # single epoch of training with x=state, y=target_f; fit decreases loss btwn target_f and y_hat
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

#### Interact with environment

In [10]:
agent = DQNAgent(state_size, action_size) # initialise agent

2024-01-15 09:34:38.528146: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-15 09:34:38.716517: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/marcel/anaconda3/lib/python3.9/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [11]:
done = False
for e in range(n_episodes):   # iterate over new episodes of the game
    state = env.reset()       # reset state at start of each new episode of the game
    state = np.reshape(state[0], [1, state_size])
    
    for time in range(5000):  # time represents a frame of the game; goal is to keep pole upright as long as possible up to range, e.g., 500 or 5000 timesteps

        action = agent.act(state)   # action is either 0 or 1 (move cart left or right); decide on one or other here
        next_state, reward, done, _, _ = env.step(action) # agent interacts with env, gets feedback; 4 state data points, e.g., pole angle, cart position        
        
        reward = reward if not done else -10           # reward +1 for each additional frame with pole upright        
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done) # remember the previous timestep's state, actions, reward, etc.        
        state = next_state     # set "current state" for upcoming iteration to the current next state        
        if done:               # episode ends if agent drops pole or we reach timestep 5000
            print("episode: {}/{}, score: {}, e: {:.2}" # print the episode's score and agent's epsilon
                  .format(e, n_episodes, time, agent.epsilon))
            break # exit loop
            
    if len(agent.memory) > batch_size:
        agent.replay(batch_size) # train the agent by replaying the experiences of the episode
        
    if e % 50 == 0:
        agent.save(output_dir + "weights_" + '{:04d}'.format(e) + ".hdf5")         

episode: 0/1001, score: 32, e: 1.0
episode: 1/1001, score: 15, e: 0.99
episode: 2/1001, score: 17, e: 0.99
episode: 3/1001, score: 80, e: 0.99
episode: 4/1001, score: 17, e: 0.98
episode: 5/1001, score: 13, e: 0.98
episode: 6/1001, score: 11, e: 0.97
episode: 7/1001, score: 9, e: 0.97
episode: 8/1001, score: 11, e: 0.96
episode: 9/1001, score: 10, e: 0.96
episode: 10/1001, score: 39, e: 0.95
episode: 11/1001, score: 14, e: 0.95
episode: 12/1001, score: 23, e: 0.94
episode: 13/1001, score: 21, e: 0.94
episode: 14/1001, score: 17, e: 0.93
episode: 15/1001, score: 14, e: 0.93
episode: 16/1001, score: 28, e: 0.92
episode: 17/1001, score: 10, e: 0.92
episode: 18/1001, score: 15, e: 0.91
episode: 19/1001, score: 24, e: 0.91
episode: 20/1001, score: 9, e: 0.9
episode: 21/1001, score: 15, e: 0.9
episode: 22/1001, score: 14, e: 0.9
episode: 23/1001, score: 31, e: 0.89
episode: 24/1001, score: 17, e: 0.89
episode: 25/1001, score: 10, e: 0.88
episode: 26/1001, score: 12, e: 0.88
episode: 27/1001,

episode: 220/1001, score: 39, e: 0.33
episode: 221/1001, score: 71, e: 0.33
episode: 222/1001, score: 37, e: 0.33
episode: 223/1001, score: 54, e: 0.33
episode: 224/1001, score: 47, e: 0.33
episode: 225/1001, score: 37, e: 0.32
episode: 226/1001, score: 31, e: 0.32
episode: 227/1001, score: 17, e: 0.32
episode: 228/1001, score: 30, e: 0.32
episode: 229/1001, score: 23, e: 0.32
episode: 230/1001, score: 35, e: 0.32
episode: 231/1001, score: 24, e: 0.31
episode: 232/1001, score: 15, e: 0.31
episode: 233/1001, score: 14, e: 0.31
episode: 234/1001, score: 13, e: 0.31
episode: 235/1001, score: 34, e: 0.31
episode: 236/1001, score: 29, e: 0.31
episode: 237/1001, score: 14, e: 0.3
episode: 238/1001, score: 21, e: 0.3
episode: 239/1001, score: 22, e: 0.3
episode: 240/1001, score: 16, e: 0.3
episode: 241/1001, score: 13, e: 0.3
episode: 242/1001, score: 15, e: 0.3
episode: 243/1001, score: 10, e: 0.3
episode: 244/1001, score: 14, e: 0.29
episode: 245/1001, score: 18, e: 0.29
episode: 246/1001, 

episode: 433/1001, score: 209, e: 0.11
episode: 434/1001, score: 340, e: 0.11
episode: 435/1001, score: 405, e: 0.11
episode: 436/1001, score: 270, e: 0.11
episode: 437/1001, score: 301, e: 0.11
episode: 438/1001, score: 339, e: 0.11
episode: 439/1001, score: 371, e: 0.11
episode: 440/1001, score: 208, e: 0.11
episode: 441/1001, score: 133, e: 0.11
episode: 442/1001, score: 184, e: 0.11
episode: 443/1001, score: 298, e: 0.11
episode: 444/1001, score: 343, e: 0.11
episode: 445/1001, score: 229, e: 0.11
episode: 446/1001, score: 417, e: 0.11
episode: 447/1001, score: 369, e: 0.11
episode: 448/1001, score: 241, e: 0.11
episode: 449/1001, score: 347, e: 0.11
episode: 450/1001, score: 274, e: 0.1
episode: 451/1001, score: 370, e: 0.1
episode: 452/1001, score: 334, e: 0.1
episode: 453/1001, score: 294, e: 0.1
episode: 454/1001, score: 335, e: 0.1
episode: 455/1001, score: 353, e: 0.1
episode: 456/1001, score: 259, e: 0.1
episode: 457/1001, score: 381, e: 0.1
episode: 458/1001, score: 285, e:

KeyboardInterrupt: 

In [ ]:
# saved agents can be loaded with agent.load("./path/filename.hdf5") 